# Problem Description
Suppose that $Y$ is a random variable taking on one of $n$ known values: $a_1, a_2, ... , a_n$. 

Suppose we know that $Y$ either has distribution $p$ given by $P(Y = a_j) = p_j$ or it has distribution $q$ given by $P(Y = a_j) = q_j$.

Of course, the numbers $p_j,\ j = 1, 2, ... , n$ are nonnegative and sum to
one. The same is true for the $q_j$ ’s. Based on a single observation of $Y$ ,
we wish to guess whether it has distribution $p$ or distribution $q$. That is,
for each possible outcome $a_j$ , we will assert with probability $x_j$ that the
distribution is $p$ and with probability $1−x_j$ that the distribution is $q$. We wish
to determine the probabilities $x_j,\ j = 1, 2,..., n$, such that the probability
of saying the distribution is $p$ when in fact it is $q$ has probability no larger
than $\beta$, where $\beta$ is some small positive value (such as 0.05). Furthermore,
given this constraint, we wish to maximize the probability that we say the
distribution is $p$ when in fact it is $p$. Formulate this maximization problem
as a linear programming problem.

In this notebook, we want to create two simple discrete distributions on the same support to work through this problem. We will formulate and solve the linear program with `pyomo`, and will take the solution and build a Monte Carlo simulation to verify that the rate of mistaking distribution $q$ for $p$ is in fact lower than or equal to our defined $\beta$.

In [1]:
import numpy as np

support = list(range(0, 10))
p_dist = [0.1] * 10
q_dist = [0.2, 0.15, 0.1, 0.05, 0, 0, 0.05, 0.1, 0.15, 0.2]

# Single draw
draw = np.random.choice(support, p=q_dist)
draw

np.int64(9)

# Linear Program

In [2]:
import pyomo.environ as pyo

In [3]:
from typing import List

def build_model(support: List[float], p_dist: List[float], q_dist: List[float], beta: float):
    m = pyo.ConcreteModel()
    m.SUPPORT = pyo.Set(initialize=support)
    m.p_dist = pyo.Param(m.SUPPORT, initialize={i: p_dist[i] for i in support})
    m.q_dist = pyo.Param(m.SUPPORT, initialize={i: q_dist[i] for i in support})
    m.prob_of_guessing_p = pyo.Var(m.SUPPORT, domain=pyo.NonNegativeReals, bounds=(0, 1))
    
    @m.Constraint()
    def mistake_q_for_p_prob_constraint(m):
        return sum(m.q_dist[j] * m.prob_of_guessing_p[j] for j in m.SUPPORT) <= beta

    @m.Objective(sense=pyo.maximize)
    def prob_of_guessing_p_right(m):
        return sum(m.p_dist[j] * m.prob_of_guessing_p[j] for j in m.SUPPORT) 

    return m


mistake_q_for_p_rate = 0.05
model = build_model(support, p_dist, q_dist, beta=mistake_q_for_p_rate)
model.pprint()

1 Set Declarations
    SUPPORT : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Param Declarations
    p_dist : Size=10, Index=SUPPORT, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :   0.1
          1 :   0.1
          2 :   0.1
          3 :   0.1
          4 :   0.1
          5 :   0.1
          6 :   0.1
          7 :   0.1
          8 :   0.1
          9 :   0.1
    q_dist : Size=10, Index=SUPPORT, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :   0.2
          1 :  0.15
          2 :   0.1
          3 :  0.05
          4 :     0
          5 :     0
          6 :  0.05
          7 :   0.1
          8 :  0.15
          9 :   0.2

1 Var Declarations
    prob_of_guessing_p : Size=10, Index=SUPPORT
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : NonNegativeRe

In [4]:
solver = pyo.SolverFactory('appsi_highs')
results = solver.solve(model)
print('Solver status:', results.Solver.status)
print('Solver termination condition:', results.Solver.termination_condition)
print('Maximal probability of guessing distribution p right:', round(pyo.value(model.prob_of_guessing_p_right), 2))

optimal_guess_p_probs = [model.prob_of_guessing_p[i].value for i in model.SUPPORT]
print("Optimal guessing probabilities:", optimal_guess_p_probs)

Solver status: ok
Solver termination condition: optimal
Maximal probability of guessing distribution p right: 0.3
Optimal guessing probabilities: [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]


# Monte Carlo Simulations

In [5]:
def draw_and_guess(support: List[float], dist: List[float], guess_p_probs: List[float]):
    draw = np.random.choice(support, p=dist)
    guess_p_prob = guess_p_probs[draw]
    is_guessing_p = np.random.binomial(1, guess_p_prob)
    return is_guessing_p

Two sources of randomness: drawing one observation from the given generating distribution ($p$ or $q$) & the random guess.

In [6]:
n_iterations = 100000

In [7]:
guesses = []
for i in range(n_iterations):
    is_guessing_p = draw_and_guess(support, p_dist, guess_p_probs=optimal_guess_p_probs)
    guesses.append(is_guessing_p)

sum(guesses) / len(guesses)

0.30092

Monte Carlo simulation shows that the guessing rule is actually 30% right, if the generating distribution is $p$. 

In [8]:
guesses = []
for i in range(n_iterations):
    is_guessing_p = draw_and_guess(support, q_dist, guess_p_probs=optimal_guess_p_probs)
    guesses.append(is_guessing_p)

sum(guesses) / len(guesses)

0.04946

Monte Carlo simulation shows that the guessing rule is actually 5% wrong, if the generating distribution is $q$.